In [ ]:
from __future__ import division
import gym
import numpy as np
import torch
from torch.autograd import Variable
import os
import psutil
import gc

import numpy as np
import pybullet_envs
import time
import train
import buffer
#import matplotlib.pyplot as plt
#%matplotlib inline
#env = gym.make('BipedalWalker-v3')
env = gym.make('InvertedPendulumSwingupBulletEnv-v0')
env.render()

env.reset()
MAX_EPISODES = 5000
MAX_STEPS = 1000
MAX_BUFFER = 1000000
MAX_TOTAL_REWARD = 300
S_DIM = env.observation_space.shape[0]
A_DIM = env.action_space.shape[0]
A_MAX = env.action_space.high[0]


In [2]:
S_DIM,  A_DIM, A_MAX

(5, 1, 1.0)

In [3]:
ram = buffer.MemoryBuffer(MAX_BUFFER)
trainer = train.Trainer(S_DIM, A_DIM, A_MAX, ram)
#render = lambda : plt.imshow(env.render(mode='rgb_array'))

In [ ]:
for _ep in range(MAX_EPISODES):
    #env.render()
    #observation = env.reset()
    print('EPISODE :- {}'.format(_ep))
    for r in range(MAX_STEPS):
        
        env.render()
        #render()
        state = np.float32(observation)

        action = trainer.get_exploration_action(state)
        # if _ep%5 == 0:
        # 	# validate every 5th episode
        # 	action = trainer.get_exploitation_action(state)
        # else:
        # 	# get action based on observation, use exploration policy here
        # 	action = trainer.get_exploration_action(state)

        new_observation, reward, done, info = env.step(action)

        # # dont update if this is validation
        # if _ep%50 == 0 or _ep>450:
        # 	continue

        if done:
            new_state = None
        else:
            new_state = np.float32(new_observation)
            # push this exp in ram
            ram.add(state, action, reward, new_state)

        observation = new_observation

        # perform optimization
        trainer.optimize()
        if done:
            break

    # check memory consumption and clear memory
    gc.collect()
    # process = psutil.Process(os.getpid())
    # print(process.memory_info().rss)

    if _ep%100 == 0:
        trainer.save_models(_ep)